<center>
<img src="../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
<center>
Автор материала: Юрий Кашницкий, программист-исследователь Mail.Ru Group <br> 

Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание 6 (демо). Линейная регрессия, Lasso и RF-регрессия в задаче по определению качества вина</center>


**Заполните пропущенный код и ответьте на вопросы в [онлайн-форме](https://docs.google.com/forms/d/1gsNxgkd0VqidZp4lh9mnCQnJw3b0IFR1C4WBES86J40).**

In [3]:
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.metrics.regression import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

**Будем работать с набором данных по качеству белого вина (репозиторий UCI).**
**Загружаем данные.**

In [4]:
data = pd.read_csv('../data/winequality-white.csv', sep=';')

In [21]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
fixed acidity           4898 non-null float64
volatile acidity        4898 non-null float64
citric acid             4898 non-null float64
residual sugar          4898 non-null float64
chlorides               4898 non-null float64
free sulfur dioxide     4898 non-null float64
total sulfur dioxide    4898 non-null float64
density                 4898 non-null float64
pH                      4898 non-null float64
sulphates               4898 non-null float64
alcohol                 4898 non-null float64
quality                 4898 non-null int64
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


**Отделите целевой признак, разделите обучающую выборку в отношении 7:3 (30% - под оставленную выборку, пусть random_state=17) и отмасштабируйте данные с помощью StandardScaler.**

In [49]:
# Ваш код здесь
y = data['quality']
X = data.drop('quality', axis=1)

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.3, random_state=17)

scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), index=X_train.index, columns=X_train.columns)
X_holdout_scaled = pd.DataFrame(scaler.transform(X_holdout), index=X_holdout.index, columns=X_holdout.columns)

In [50]:
data.shape

(4898, 12)

In [51]:
X_train.shape, X_holdout.shape, y_train.shape, y_holdout.shape

((3428, 11), (1470, 11), (3428,), (1470,))

In [52]:
X_train.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
96,6.0,0.34,0.66,15.9,0.046,26.0,164.0,0.99790,3.14,0.50,8.8
2292,6.5,0.32,0.23,1.2,0.054,39.0,208.0,0.99272,3.18,0.46,9.9
1054,7.0,0.31,0.52,1.7,0.029,5.0,61.0,0.99180,3.07,0.43,10.4
3687,6.0,0.16,0.27,12.0,0.030,39.0,98.0,0.99402,3.15,0.34,10.8
596,6.9,0.41,0.33,10.1,0.043,28.0,152.0,0.99680,3.20,0.52,9.4


In [53]:
X_train_scaled.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
96,-0.992730,0.583755,2.641169,1.924266,-0.000505,-0.544363,0.599730,1.341727,-0.321001,0.103009,-1.411199
2292,-0.411415,0.387471,-0.844072,-1.030761,0.346034,0.207260,1.624343,-0.428859,-0.055511,-0.253133,-0.517292
1054,0.169901,0.289329,1.506439,-0.930250,-0.736902,-1.758523,-1.798796,-0.743326,-0.785610,-0.520239,-0.110970
3687,-0.992730,-1.182798,-0.519864,1.140279,-0.693584,0.207260,-0.937190,0.015496,-0.254629,-1.321558,0.214087
596,0.053638,1.270748,-0.033551,0.758337,-0.130458,-0.428729,0.320290,0.965733,0.077235,0.281080,-0.923613


## Линейная регрессия

**Обучите простую линейную регрессию.**

In [57]:
# Ваш код здесь
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

**<font color='red'>Вопрос 1:</font> Каковы среднеквадратичные ошибки линейной регрессии на обучающей и отложенной выборках?**

In [62]:
# Ваш код здесь
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, linreg.predict(X_train_scaled)))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, linreg.predict(X_holdout_scaled)))

Mean squared error (train): 0.558
Mean squared error (test): 0.584


**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина (учтите, что большие по модулю отрицательные значения коэффициентов тоже говорят о сильном влиянии). Создайте для этого новый небольшой DataFrame.**<br>
**<font color='red'>Вопрос 2:</font> Какой признак линейная регрессия считает наиболее сильно влияющим на качество вина?**

In [87]:
# Ваш код здесь
linreg_coef = pd.DataFrame(linreg.coef_, index=X_train.columns, columns=['linreg_coef'])
linreg_coef['linreg_coef'].abs().sort_values(ascending=False)

density                 0.665720
residual sugar          0.538164
volatile acidity        0.192260
pH                      0.150036
alcohol                 0.129533
fixed acidity           0.097822
sulphates               0.062053
free sulfur dioxide     0.042180
total sulfur dioxide    0.014304
chlorides               0.008127
citric acid             0.000183
Name: linreg_coef, dtype: float64

## Lasso-регрессия

**Обучите Lasso-регрессию с небольшим коэффициентом $\alpha = 0.01$ (слабая регуляризация). Пусть опять random_state=17.**

In [88]:
# Ваш код здесь
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=17,
   selection='cyclic', tol=0.0001, warm_start=False)

**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина. Какой признак "отвалился" первым, то есть наименее важен для объяснения целевого признака в модели Lasso?**

In [91]:
# Ваш код здесь
lasso1_coef = pd.DataFrame(lasso1.coef_, index=X_train.columns, columns=['lasso1_coef'])
lasso1_coef['lasso1_coef'].abs().sort_values(ascending=False)

alcohol                 0.322425
residual sugar          0.256363
density                 0.235492
volatile acidity        0.188479
pH                      0.067277
free sulfur dioxide     0.043088
sulphates               0.029722
chlorides               0.002747
total sulfur dioxide    0.000000
citric acid             0.000000
fixed acidity           0.000000
Name: lasso1_coef, dtype: float64

**Теперь определите лучшее значение $\alpha$ в процессе кросс-валидации 5-кратной кросс-валидации. Используйте LassoCV и random_state=17.**

In [134]:
# Ваш код здесь
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(n_jobs=-1, random_state=17, cv=5, alphas=alphas)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000e-06, 1.09699e-06, ..., 9.11589e+01, 1.00000e+02]),
    copy_X=True, cv=5, eps=0.001, fit_intercept=True, max_iter=1000,
    n_alphas=100, n_jobs=-1, normalize=False, positive=False,
    precompute='auto', random_state=17, selection='cyclic', tol=0.0001,
    verbose=False)

In [135]:
lasso_cv.alpha_

0.0002833096101839324

**Выведите коэффициенты "лучшего" Lasso в порядке убывания влияния на качество вина.**<br>
**<font color='red'>Вопрос 3:</font> Какой признак "обнулился первым" в настроенной модели LASSO?**

In [97]:
# Ваш код здесь
lasso_cv_coef = pd.DataFrame(lasso_cv.coef_, index=X_train.columns, columns=['lasso_cv_coef'])
lasso_cv_coef['lasso_cv_coef'].abs().sort_values(ascending=False)

density                 0.641542
residual sugar          0.522635
volatile acidity        0.191982
pH                      0.145230
alcohol                 0.139980
fixed acidity           0.091608
sulphates               0.060524
free sulfur dioxide     0.042894
total sulfur dioxide    0.012472
chlorides               0.006492
citric acid             0.000000
Name: lasso_cv_coef, dtype: float64

**Оцените среднеквадратичную ошибку модели на обучающей и тестовой выборках.**<br>
**<font color='red'>Вопрос 4:</font> Каковы среднеквадратичные ошибки настроенной LASSO-регрессии на обучающей и отложенной выборках?**

In [98]:
# Ваш код здесь
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, lasso_cv.predict(X_train_scaled)))
print("Mean squared error (test): %.3f" %  mean_squared_error(y_holdout, lasso_cv.predict(X_holdout_scaled)))

Mean squared error (train): 0.558
Mean squared error (test): 0.583


## Случайный лес

**Обучите случайный лес с параметрами "из коробки", фиксируя только random_state=17.**

In [99]:
# Ваш код здесь
forest = RandomForestRegressor(n_jobs=-1, random_state=17) 
forest.fit(X_train_scaled, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=17, verbose=0, warm_start=False)

**<font color='red'>Вопрос 5:</font> Каковы среднеквадратичные ошибки случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error' и остальными параметрами по умолчанию) и на отложенной выборке?**

In [115]:
# Ваш код здесь
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, forest.predict(X_train_scaled)))
print("Mean squared error (cv): %.3f" % 
    -cross_val_score(forest, X_train_scaled, y_train, scoring='neg_mean_squared_error').mean())
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, forest.predict(X_holdout_scaled)))

Mean squared error (train): 0.075
Mean squared error (cv): 0.460
Mean squared error (test): 0.421


**Настройте параметры min_samples_leaf и max_depth с помощью GridSearchCV и опять проверьте качество модели на кросс-валидации и на отложенной выборке.**

In [125]:
forest_params = {'max_depth': list(range(10, 25)), 
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6,12))}
# Ваш код здесь
locally_best_forest = GridSearchCV(forest, forest_params, n_jobs=-1)
locally_best_forest.fit(X_train_scaled, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=17, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24], 'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7], 'max_features': [6, 7, 8, 9, 10, 11]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [126]:
locally_best_forest.best_params_, locally_best_forest.best_score_

({'max_depth': 22, 'max_features': 7, 'min_samples_leaf': 1},
 0.42345378900629316)

**К сожалению, результаты  GridSearchCV не полностью воспроизводимы (могут отличаться на разных платформах даже при фиксировании *random_state*). Поэтому обучите лес с параметрами max_depth=19, max_features=7, и min_samples_leaf=1 (лучшие в моем случае).**<br>
**<font color='red'>Вопрос 6:</font> Каковы среднеквадратичные ошибки настроенного случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error') и на отложенной выборке?**

In [127]:
# Ваш код здесь
best_forest = RandomForestRegressor(max_depth=19, max_features=7, min_samples_leaf=1, n_jobs=-1, random_state=17)
best_forest.fit(X_train_scaled, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=19,
           max_features=7, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=-1, oob_score=False, random_state=17,
           verbose=0, warm_start=False)

In [129]:
# Ваш код здесь
print("Mean squared error (cv): %.3f" % 
      -cross_val_score(best_forest, X_train_scaled, y_train, scoring='neg_mean_squared_error').mean())
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, best_forest.predict(X_holdout_scaled)))

Mean squared error (cv): 0.457
Mean squared error (test): 0.410


In [130]:
# Мой результат
my_best_forest = RandomForestRegressor(max_depth=22, max_features=7, min_samples_leaf=1, n_jobs=-1, random_state=17)
my_best_forest.fit(X_train_scaled, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=22,
           max_features=7, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=-1, oob_score=False, random_state=17,
           verbose=0, warm_start=False)

In [131]:
print("Mean squared error (cv): %.3f" % 
      -cross_val_score(my_best_forest, X_train_scaled, y_train, scoring='neg_mean_squared_error').mean())
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, my_best_forest.predict(X_holdout_scaled)))

Mean squared error (cv): 0.453
Mean squared error (test): 0.415


**Оцените важность признаков с помощью случайного леса.**<br>
**<font color='red'>Вопрос 7:</font> Какой признак оказался главным в настроенной модели случайного леса?**

In [133]:
# Ваш код здесь
rf_importance = pd.DataFrame(best_forest.feature_importances_, index=X_train.columns, columns=['feature_importances'])
rf_importance['feature_importances'].abs().sort_values(ascending=False)

alcohol                 0.224432
volatile acidity        0.119393
free sulfur dioxide     0.116147
pH                      0.072806
total sulfur dioxide    0.071318
residual sugar          0.070160
density                 0.069367
chlorides               0.067982
fixed acidity           0.064268
citric acid             0.062945
sulphates               0.061184
Name: feature_importances, dtype: float64

# 10/10